In [7]:
pip install pandas patool

Note: you may need to restart the kernel to use updated packages.


In [43]:
#Extraction 

import os
import patoolib

# File paths
file_path = r"D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\data.rar"
extract_path = r"D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Extracted_data"

# Ensure the extraction folder exists
os.makedirs(extract_path, exist_ok=True)

# Check if RAR file exists
if os.path.exists(file_path):
    try:
        # Extract the RAR file
        patoolib.extract_archive(file_path, outdir=extract_path)
        print("Extraction completed successfully.")
    
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print(f"Error: RAR file not found at {file_path}")

INFO patool: Extracting D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\data.rar ...
INFO patool: running "C:\Program Files\WinRAR\rar.EXE" x -kb -or -- D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\data.rar
INFO patool: ... D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\data.rar extracted to `D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Extracted_data'.


Extraction completed successfully.


In [73]:
import glob
import pandas as pd
import yaml
import os

output_dir = r"D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV"

os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists

yaml_data = []  # List to store DataFrames

# Loop through each YAML file in extracted folders
for folder in glob.glob(os.path.join(extract_path, "20*")):
    for yaml_file in glob.glob(os.path.join(folder, "*.yaml")):
        try:
            with open(yaml_file, "r") as f:
                data = yaml.safe_load(f)  # Load YAML safely

            if data:  # Ensure the file contains valid data
                df = pd.DataFrame(data)  
                yaml_data.append(df)
            else:
                print(f"Warning: {yaml_file} is empty or invalid.")
        
        except Exception as e:
            print(f"Unexpected error processing {yaml_file}: {e}")
print(f"Successfully stored the yaml_data in list")
# Combine all extracted data
try:
    if yaml_data:
        df_final = pd.concat(yaml_data, ignore_index=True)

        # Ensure "Ticker" column exists before grouping
        if "Ticker" in df_final.columns:
            for ticker, df_group in df_final.groupby("Ticker"):
                try:
                    csv_file_path = os.path.join(output_dir, f"{ticker}.csv")
                    df_group.to_csv(csv_file_path, index=False)
                    print(f"Successfully Saved: {csv_file_path}")
                except Exception as e:
                    print(f"Error saving {ticker}.csv: {e}")

            print("All ticker CSV files saved successfully.")
        else:
            print("Error: 'Ticker' column missing in extracted data.")
    else:
        print("YAML data not found.")

except Exception as e:
    print(f"Unexpected error during data processing: {e}")

Successfully appended
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\ADANIENT.csv
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\ADANIPORTS.csv
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\APOLLOHOSP.csv
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\ASIANPAINT.csv
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\AXISBANK.csv
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\BAJAJ-AUTO.csv
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\BAJAJFINSV.csv
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\BAJFINANCE.csv
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\BEL.csv
Successfully Saved: D:\Guvi_Project\DD_Stock_Analysis\Data_Folder\Processed_CSV\BHARTIARTL.csv
Successfully Saved: D:\Guvi_Project\DD_